In [3]:
# Import public packages and functions
import os
import pandas as pd
import numpy as np
import sys
import json
import pickle
from pathlib import Path
import seaborn as sns

# inserting the lib folder to the compiler
sys.path.insert(0, './lib')
sys.path.insert(0, './utils/')

import warnings
warnings.filterwarnings("ignore")

from lib_event import EVENTS
from lib_data import DATA_IO

import utils_plotting_accelerometer, utils_misc

In [5]:
PATH_CURR   = os.path.abspath(os.curdir)    # current code
PATH        = (str(Path(PATH_CURR).parent)) # data repository: upper directory where datasets situated
PATH_DATA   = PATH + "\\data"               # the recordings data directory
PATH_EVENTS = PATH + "\\events"
SUB_LIST    = utils_misc.get_SUB_list(PATH_DATA) # get the SUB id list which we have a recording of them

# 1. ACCELEROMETER EVENTS

## 1.1 Detect Accelerometer Events

In [ ]:
for SUB in SUB_LIST:

    # get event history of the patient
    EVENTS_HISTORY    = EVENTS(PATH, SUB, 'acc_right')

    # get the tapping indices of SUB
    with open(DATA_IO.path_events + "taps_indices/" + SUB + "_taps", 'rb') as handle:
        SUB_tap_indices = pickle.load(handle)
        
    df_SUB_acc_events = EVENTS_HISTORY.get_event_dataframe(SUB_tap_indices)
    df_SUB_acc_events = df_SUB_acc_events[df_SUB_acc_events.duration >= 0.2]
    df_SUB_acc_events.to_csv(DATA_IO.path_events + "SUB_"+ SUB +"_EVENTS_ACC.csv", index=False)

## 1.2  Merge Accelerometer Events

In [ ]:
df_accelerometer_events = pd.DataFrame()
event_files             = utils_misc.get_files_with_specific_format(PATH_EVENTS, suffix=".csv")

for file in event_files:
    if("ACC" in file):
        if(len(df_accelerometer_events)==0): # if the dataframe is empty
            df_accelerometer_events = pd.read_csv(PATH_EVENTS + "\\" + file)
        else:
            df_accelerometer_events = pd.concat([df_accelerometer_events, pd.read_csv(PATH_EVENTS + "\\" + file)], ignore_index=True)

# Apply the function to each row and create a new column
df_accelerometer_events.to_csv(DATA_IO.path_events + "EVENTS_ACC.csv", index=False)

# 2. LFP/ECOG EVENTS

In [7]:
def find_closest_index(time_array, target_time):
    """
    Finds the index of the closest time value in a sorted array.

    Parameters:
    time_array (np.array): Sorted array of time values in float.
    target_time (float): The target time value to search for.

    Returns:
    int: The index of the closest time value in the array.
    """
    # Use searchsorted to find the insertion point
    idx = np.searchsorted(time_array, target_time)

    # Handle edge cases
    if idx == 0:
        return 0
    elif idx == len(time_array):
        return len(time_array) - 1
    else:
        # Compare the two closest values (left and right of the insertion point)
        left_idx = idx - 1
        right_idx = idx
        if abs(time_array[left_idx] - target_time) <= abs(time_array[right_idx] - target_time):
            return left_idx
        else:
            return right_idx

In [9]:
df_accelerometer_events = pd.read_csv(DATA_IO.path_events + "EVENTS_ACC.csv")
df_LFP_events           = pd.DataFrame(columns=df_accelerometer_events.columns)

for SUB in SUB_LIST:

    # get patients LFP EVENT history
    EVENTS_LFP = EVENTS(PATH, SUB, 'lfp_right')

    # get time array 
    SUB_time   = EVENTS_LFP.times/60

    for index, row in df_accelerometer_events[df_accelerometer_events.patient==int(SUB)].iterrows():
        tap                       = row.copy()
        tap["event_start_index"]  = find_closest_index(SUB_time, row.copy().event_start_time)
        tap["event_finish_index"] = find_closest_index(SUB_time, row.copy().event_finish_time)
        tap["event_start_time"]   = SUB_time[tap["event_start_index"]]
        tap["event_finish_time"]  = SUB_time[tap["event_finish_index"]]
        tap["duration"]           = (tap["event_finish_index"]-tap["event_start_index"])/EVENTS_LFP.fs
        df_LFP_events.loc[len(df_LFP_events)] = tap   

# Apply the function to each row and create a new column
df_LFP_events = df_LFP_events[df_LFP_events.duration>=0.2]
df_LFP_events.to_csv(DATA_IO.path_events + "EVENTS_LFP.csv", index=False)

EVENT HISTORY: SUB-008
... loading started
... pickle loading: C:\Users\a.kaymak\Desktop\Papers\2025 Parkinson STN-DBS Dyskinesia LFP-ECG\files\data\sub-008\008_mergedData_v4.0_lfp_right.P
... task periods were defined
... events were categorized
... dyskinesia evaluation was collected
... event loading completed
--------------------------------------------------------------------
EVENT HISTORY: SUB-009
... loading started
... pickle loading: C:\Users\a.kaymak\Desktop\Papers\2025 Parkinson STN-DBS Dyskinesia LFP-ECG\files\data\sub-009\009_mergedData_v4.0_lfp_right.P
... task periods were defined
... events were categorized
... dyskinesia evaluation was collected
... event loading completed
--------------------------------------------------------------------
EVENT HISTORY: SUB-010
... loading started
... pickle loading: C:\Users\a.kaymak\Desktop\Papers\2025 Parkinson STN-DBS Dyskinesia LFP-ECG\files\data\sub-010\010_mergedData_v4.0_lfp_right.P

	...correct tasks in C:\Users\a.kaymak\Des